In [3]:
# Import packages

import sqlite3
import requests
import pandas as pd

In [4]:
"""
This function retun holidays of the year by getting them from date.nager.at api
input:
    year as string like: '2022'
return:
    list of holiday days:
    ['2023-01-01', '2023-01-06', '2023-04-07']
"""

def get_holidays(year):
    response = requests.get(f'https://date.nager.at/api/v3/publicholidays/{year}/fi')
    if response.status_code == 200:
        holidays_data = response.json()
        return [holiday['date'] for holiday in holidays_data]
    else:
        raise 'site not response'

In [5]:
holiday_dates_2022 = get_holidays(2022)
holiday_dates_2023 = get_holidays(2023)

In [7]:
holiday_dates_2023

['2023-01-01',
 '2023-01-06',
 '2023-04-07',
 '2023-04-09',
 '2023-04-10',
 '2023-05-01',
 '2023-05-18',
 '2023-05-28',
 '2023-06-23',
 '2023-06-24',
 '2023-11-04',
 '2023-12-06',
 '2023-12-24',
 '2023-12-25',
 '2023-12-26']